# 进阶操作

在本节中，我们将讨论一些主题，这些主题对于编写模块可能不是必需的，但绝对值得考虑。


## 测试脚本

为了表明代码能够按预期工作并确保将来继续工作，应该为新模块编写一个测试，可以使用Python或Bash将此类测试编写为通用脚本。

在Python中的[doctest](https://docs.python.org/2/library/doctest.html)可以用于对代码中的部分模块进行测试。

但是，在GRASS GIS中，其专用的Python测试包为[gunittest](http://grass.osgeo.org/grass71/manuals/libpython/gunittest_testing.html)其中包含使GRASS GIS模块测试尽可能方便的功能。

`grass.gunittest`包在‘trunk’版本中可用，版本信息可查看[development version](http://grass.osgeo.org/download/software/sources/)

测试是作为类的函数（方法）编写的，该类是从`grass.gunittest.case`包中的`TestCase`继承的。测试函数名称必须以`test_`开头，`test_`函数可以有一个或多个。`runModule（）`函数用于测试不直接调用的模块。使用`assertModule（）`函数以给定的参数调用正在测试的模块，并且期望过程能够成功完成。在每次调用test_函数之前和之后都会调用`setUp（）`和`tearDown（）`函数，应该始终删除创建的地图（使用`g.remove`）。

In [1]:
%%python
# Only available for GRASS GIS 7.1 (and above)
from grass.gunittest.case import TestCase
from grass.gunittest.main import test
import grass.script as gscript

class TestViewshedPoint(TestCase):

    prefix = 'test_rviewshedpoints_'

    @classmethod
    def setUpClass(cls):
        """Ensure expected computational region and create input vector points"""
        cls.use_temp_region()
        cls.runModule('g.region', n=225200, s=222500, w=637500, e=640000, raster='elevation')
        cls.runModule('v.random', output=cls.prefix + 'input_points', npoints=10, seed=2)

    def setUp(self):
        """Create r.viewshed.points output for every test method"""
        self.runModule('r.viewshed.points', elevation='elevation',
                       points=self.prefix + 'input_points', output_points=self.prefix + 'output_points',
                       viewshed_basename=self.prefix + 'output_viewshed')

    def tearDown(self):
        """Remove r.viewshed.points output after every test"""
        self.runModule('g.remove', type=['raster', 'vector'], pattern=self.prefix + '*', flags='f')

    def test_outputs_exists(self):
        """Test that output maps exist"""
        num_viewsheds = len(gscript.list_grouped(['raster'],
                                                 pattern=self.prefix + '*')[gscript.gisenv()['MAPSET']])
        self.assertEqual(first=10, second=num_viewsheds, msg="Wrong number of viewsheds computed")
        self.assertVectorExists(self.prefix + 'output_points')
        
    def test_area_matches(self):
        """Test if areas are correct"""
        minmax = {'min': 392800, 'max': 1759000}
        self.assertVectorFitsUnivar(map=self.prefix + 'output_points', column='area',
                                    reference=minmax, precision=1e-6)


if __name__ == '__main__':
    test()


OSError error(2): No such file or directory
OSError error(2): No such file or directory


EE
ERROR: test_area_matches (__main__.TestViewshedPoint)
Test if areas are correct
----------------------------------------------------------------------
Traceback (most recent call last):
  File "<stdin>", line 21, in setUp
  File "/usr/lib/grass76/etc/python/grass/gunittest/case.py", line 1074, in runModule
    module = _module_from_parameters(module, **kwargs)
  File "/usr/lib/grass76/etc/python/grass/gunittest/case.py", line 1205, in _module_from_parameters
    module = SimpleModule(module, **kwargs)
  File "/usr/lib/grass76/etc/python/grass/gunittest/gmodules.py", line 54, in __init__
    Module.__init__(self, cmd, *args, **kargs)
  File "/usr/lib/grass76/etc/python/grass/pygrass/modules/interface/module.py", line 530, in __init__
    raise GrassError(str_err % self.name)
GrassError: Error running: `r.viewshed.points --interface-description`.

ERROR: test_outputs_exists (__main__.TestViewshedPoint)
Test that output maps exist
-------------------------------------------------------

## 创建GUI工具箱

可以将新的Python模块放入GRASS GIS的标准图形用户界面中，GUI工具箱是一种在wxGUI菜单中自定义项目的方法，只需要生成一个XML文件并将其复制到预定义目录即可，XML文件可能如下所示：

In [2]:
%%file toolboxes.xml
<?xml version="1.0" encoding="UTF-8"?>
<toolboxes>
  <toolbox name="Viewsheds">
    <label>Viewsheds</label>
    <items>
      <module-item name="r.viewshed.points">
        <label>Compute special viewsheds</label>
      </module-item>
      <module-item name="r.viewshed">
        <label>Compute standard viewshed</label>
      </module-item>
      <module-item name="g.region">
        <label>Set region</label>
      </module-item>
      <module-item name="v.random">
        <label>Generate random points</label>
      </module-item>
      <separator/>
    </items>
  </toolbox>
</toolboxes>

Writing toolboxes.xml


必须将此文件`toolboxes.xml`复制到主目录中的GRASS GIS配置目录（例如，Linux上的`〜/ .grass7 / toolboxes / toolboxes.xml`）。 下次启动GRASS GIS GUI（`g.gui`）时，在“图层管理器”窗口的“搜索模块”选项卡的模块树中的“用户定义的工具箱”下看到新的工具箱。

## GRASS GIS社区

如有疑问，最好的询问地点是在[grass-user](http://lists.osgeo.org/listinfo/grass-user)邮件列表上，这也是宣布新插件的好地方。

如果您正在开发新的插件，并且需要开发人员的帮助，则可以编写[grass-dev](http://lists.osgeo.org/listinfo/grass-dev)邮件列表。

还有一些本地化的GRASS GIS邮件列表，您可以在[dedicated mailing list web page](http://grass.osgeo.org/support/mailing-lists/)上找到它们。

[Google+](https://plus.google.com/communities/111147786674687562495)上还有一个GRASS GIS社区正在等待您与GRASS GIS相关的新闻和一个[Facebook group](https://www.facebook.com/groups/96121775724/)。


## 下一步

我们正在寻找具有各种技能的人员来改善GRASS GIS项目：我们正在寻找翻译人员，网站设计师，网站内容管理员，程序员，教学人员和课程人员，以及GRASS GIS倡导者。

您也可以帮助我们[sponsoring](http://grass.osgeo.org/support/our-sponsors/) GRASS GIS（也欢迎小额捐款！），也可以[report errors or enhancement wishes](http://grass.osgeo.org/development/bug-tracking/)参与其中！